# Sentiment Analysis Using RNN

In [1]:
import numpy as np
from string import punctuation
from collections import Counter, OrderedDict
import itertools

import torch 
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

import nltk
from nltk.corpus import stopwords

In [2]:
with open('deep-learning-v2-pytorch/sentiment-analysis-network/reviews.txt', 'r') as f:
    reviews = f.read()
with open('deep-learning-v2-pytorch/sentiment-analysis-network/labels.txt', 'r') as f:
    labels = f.read()

In [4]:
# This function will recieve the imported reviews (ch by ch) and return 
def clean_text(text):
    ''' This Function recieves reviews (ch by ch) and returns a list of
    reviews without punctuation and stopwords'''
    # remove punctuation
    s = ''.join(ch.lower() for ch in text if ch not in punctuation)
    
    # separate each review and add to a list so that I have a list of reviews
    separated_reviews = []

    for review in s.split('\n'):
        review = ''.join(review)
        separated_reviews.append(review)
        
        
    # remove stopwords and return a list of reviews
    clean_text = []
    for review in separated_reviews:
        review_no_stopwords = []
        for word in review.split():
            if word not in stopwords.words('english'):
                review_no_stopwords.append(word)
        clean_text.append(' '.join(review_no_stopwords))
        
    return clean_text

In [ ]:
# map each word to a number 

In [5]:
reviews = clean_text(reviews)

In [6]:
reviews_copy = reviews

In [7]:
class ReviewEncoder:
    def __init__(self):
        self.__words_dict = {}
        self.__indexer = 1
    def encode(self, text):
        encoded_review = []
        words = text.split()
        #print(words)
        for word in words:
            if word in self. __words_dict:
                encoded_review.append(self.__words_dict[word])
            else:
                self.__words_dict[word] = self.__indexer
                self.__indexer += 1
                encoded_review.append(self.__words_dict[word])
        return encoded_review

In [10]:
encoder = ReviewEncoder()

In [17]:
encoded_reviews = []
for review in reviews:
    encoded_reviews.append(encoder.encode(review))

In [18]:
encoded_labels = encoder.encode(labels)

In [14]:
# create a function to check for reviews with length zero and dropping them

def drop_empty_reviews(text):
    
    full_reviews = []
    
    for index, review in enumerate(text):
        if len(review) != 0:
            full_reviews.append(review)
            
    return full_reviews

In [54]:
reviews = drop_empty_reviews(encoded_reviews)

In [70]:
def padding_truncation(encoded_review_list):
    
    padded_review = []
    for review in sample:
        if len(review) < 200:
            padding = 200 - len(review)
            review = ([0]*padding + review)
            padded_review.append(review)
        elif len(review) > 200:
            truncate = len(review) - 200
            review = review[truncate:]
            padded_review.append(review)
        else:
            padded_review.append(review)
            
    return padded_review

In [71]:
padded_reviews = padding_truncation(encoded_reviews)

In [45]:
# Defining training, validation and testing sets

training = int(len(reviews) * 0.8)
validation = int(training + len(reviews)*0.1)

train_x = encoded_reviews[:training]
train_y = encoded_labels[:training]

val_x = encoded_reviews[training:validation]
val_y = encoded_labels[training:validation]

test_x = encoded_reviews[validation:]
test_y = encoded_labels[validation:]

In [50]:
x = np.array(train_x)

In [51]:
x.shape

(20000,)